In [76]:
import numpy as np
import re

from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

import pandas as pd
import datetime



In [77]:
def CreerUrlParSalleParJour(salle, *args_jour):
    main_url = 'https://www.allocine.fr'
    #https://www.allocine.fr/seance/d-10/salle_gen_csalle=C0072.html
    partie_salle = 'salle_gen_csalle=' + salle
    full_url = ['/'.join([main_url, 'seance', 'd-' + str(jour), partie_salle]) + '.html' for jour in args_jour]
    return full_url

In [78]:
#Création des URLs pour les différents jours: de 0 à +10 jours
list_code_cinema = {'Ecoles Cinéma Club': 'C0071'}
                    #'Christine Cinéma Club': 'C0015'}
                    #'Filmothèque du Quartier Latin': 'C0020'}

def ObtenirDate(decalage):
    date_courante = datetime.datetime.now()
    duree = datetime.timedelta(days=int(decalage))
    date_futur = date_courante + duree
    return date_futur

def CreationListeUrlParCineParJour(list_code_cinema, nombre_jours_max):
    list_jour = np.arange(nombre_jours_max)
    dic_par_cine_par_jour = {}
    for cine, code_cine in list_code_cinema.items():
        list_par_cine = {ObtenirDate(j) : CreerUrlParSalleParJour(code_cine, j) for j in list_jour}
        dic_par_cine_par_jour[cine] = list_par_cine
    return dic_par_cine_par_jour

In [79]:
def ParsingPage(url, wait_duration = 10):
    # Remplacez ceci par le chemin vers votre pilote de navigateur
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(wait_duration)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

In [80]:
def RecupererInformationFilm(bloc_parse_film, cine , date):
    nom_film = bloc_parse_film.find('a', class_='meta-title-link', href=True).text
    print(nom_film)
    #<span class="showtimes-hour-item-value">13:50</span>
    heure_film = bloc_parse_film.find('span', class_='showtimes-hour-item-value').text #A convertir en date
    #<img alt="Un Dollar Entre les Dents" class="thumbnail-img" src="//fr.web.img2.acsta.net/r_200_283/pictures/23/07/11/16/26/2685668.jpg" width="160" height="231">
    try:
        adresse_miniature_affiche = bloc_parse_film.find('img', class_='thumbnail-img')['data-src']
    except:
        adresse_miniature_affiche = bloc_parse_film.find('img', class_='thumbnail-img')['src']
    #class="meta-body-item meta-body-info"
    #annee_du_film = bloc_parse_film.find('div', class_='meta-body-item meta-body-info').find('span', class_="date").text
    #print(annee_du_film)
    #Réalisateur(s): penser au cas où il y en aurait plusieurs
    #Pays du film: penser au cas où il y en aurait plusieurs
    #5 principaux acteurs
    #Durée du film
    pattern = r"(\b)(\d)h (\d{1,2})min\b"
    duration = bloc_parse_film.find('div', class_='meta-body-item meta-body-info')
    resultat = re.search(pattern, duration.text)
    if resultat is not None:
        heure_en_min = int(resultat.group(2))*60
        minutes_total = int(resultat.group(3)) + heure_en_min
    else:
        minutes_total = 0

    #Résumé content-txt
    resume = bloc_parse_film.find('div', class_='content-txt').text
    #Liens vers la page du film
    lien_page_film = '/'.join(['https://www.allocine.fr', bloc_parse_film.find('a', class_='meta-title-link')['href']])
    soup_film = ParsingPage(lien_page_film, 1)
    #real = soup_film.find('div', class_='meta-body-item meta-body-direction')
    #print(real)
    list_info_film = {
        'titre': nom_film,
        'horaire': heure_film,
        'url_affiche': adresse_miniature_affiche,
        #'real': real,
        'duree': minutes_total,
        'resume': resume,
        'cine': cine,
        'jour': date.strftime('%d/%m/%Y')
    }
    return list_info_film

In [81]:
def RecupererInformationsJourFilm(url, cine , date):
    soup = ParsingPage(url)
    tableau_tmp = []
    list_donnees_films_jour = soup.find_all('div', class_='card entity-card entity-card-list movie-card-theater cf hred')
    tableau_tmp = [RecupererInformationFilm(bloc, cine, date) for bloc in list_donnees_films_jour]
    if False in tableau_tmp:
        soup = ParsingPage(url)
        list_donnees_films_jour = soup.find_all('div', class_='card entity-card entity-card-list movie-card-theater cf hred')
        tableau_tmp = [RecupererInformationFilm(bloc, cine, date) for bloc in list_donnees_films_jour]
    return tableau_tmp

In [82]:
def img_tag(url):
    url = url.lstrip('/')
    index = url.find('http')
    if index == -1:
        url = "http://" + url
    return f'<img src="{url}" width="80"/>'

In [83]:
liste_par_cine_par_jour = CreationListeUrlParCineParJour(list_code_cinema, 1)

In [84]:
list_complete = []
for elem_key, elem_value in liste_par_cine_par_jour.items():
    list_complete += [RecupererInformationsJourFilm(value[0], elem_key, key) for key, value in elem_value.items()]
print(list_complete)

Apocalypse Now Final Cut
Enter the Void


KeyboardInterrupt: 

In [ ]:
from itertools import chain

print('Début')
liste_par_cine_par_jour = CreationListeUrlParCineParJour(list_code_cinema, 1)

list_complete = []
for elem_key, elem_value in liste_par_cine_par_jour.items():
    list_complete += [RecupererInformationsJourFilm(value[0], elem_key, key) for key, value in elem_value.items()]




Début
Apocalypse Now Final Cut
Enter the Void
Plein soleil
Furie au Missouri
La Vengeance est un plat qui se mange froid
Le Rouge est mis
Le Loup de Wall Street
Le Samouraï
La collectionneuse
Minuit à Paris
Ténèbres
Thelma et Louise
Phantom of the paradise
Citizen Kane
Johnny Guitare
Roberto la douceur
Django
Cinema Paradiso
Le Fils du désert
Mean Streets
Pat Garrett et Billy le Kid
Shining
Blue Velvet
2001 : l'odyssée de l'espace
Aguirre, la colère de Dieu
Les Sentiers de la gloire
Les 400 coups
À bout de souffle
In the Mood for Love
Vivre sa vie: Film en douze tableaux
Ma nuit chez Maud
Punch-Drunk Love
Stalker
The Grand Budapest Hotel


KeyError: "['Date'] not found in axis"

In [ ]:
liste_1d = list(chain(*list_complete))
df_cine = pd.DataFrame(liste_1d)
df_cine['url_affiche'] = df_cine['url_affiche'].apply(img_tag)
df_cine.sort_values(by='jour', inplace=True)

final_html = ''
for jour, group in df_cine.groupby('jour'):
    date_header = f"<h2>{jour}</h2>"
    # Supprimer la colonne "Date" avant de générer le HTML pour le tableau
    table_html = group.drop(columns='jour').sort_values(by='horaire').to_html(escape=False, index=False)
    final_html += date_header + table_html

#table_html = df_cine.to_html(escape=False, index=False)

with open('test2.html', 'w') as file_out:
    file_out.write(final_html)

print('Fin')

Fin


In [90]:
#test nom real
url_film = 'https://www.allocine.fr/film/fichefilm_gen_cfilm=4916.html'
soup_film = ParsingPage(url_film, 1)
real = soup_film.find('div', class_='meta-body-item meta-body-direction')
print()
real_2 = real.find('a', class_='xXx blue-link').text
print(real_2)


René Clément


In [91]:
acteurs = soup_film.find('div', class_='meta-body-item meta-body-actor')
acteurs_2 = acteurs.find_all('a', class_='xXx')
tableau_acteur = [act.text for act in acteurs_2]
print(tableau_acteur)

['Alain Delon', 'Marie Laforêt', 'Maurice Ronet']


In [103]:
#test nom real
url_film = 'https://www.allocine.fr/film/fichefilm-4916/casting/'
soup_film = ParsingPage(url_film, 1)
real = soup_film.find('section', class_='section casting-actor')
print(real)




<section class="section casting-actor">
<div class="titlebar"><h2 class="titlebar-title titlebar-title-md">Acteurs et actrices</h2></div>
<div class="gd gd-gap-15 gd-xs-2 gd-s-4">
<div class="card person-card person-card-col">
<figure class="thumbnail">
<a class="xXx thumbnail-container thumbnail-link" href="/personne/fichepersonne_gen_cpersonne=1920.html" title="Alain Delon">
<img alt="Alain Delon" class="thumbnail-img" data-src="https://fr.web.img6.acsta.net/c_162_216/pictures/19/05/20/14/06/2943212.jpg" height="216" src="data:image/gif;base64,R0lGODlhAwAEAIAAAAAAAAAAACH5BAEAAAAALAAAAAADAAQAAAIDhI9WADs=" width="162"/>
</a>
</figure>
<div class="meta">
<div class="meta-title">
<a class="meta-title-link" href="/personne/fichepersonne_gen_cpersonne=1920.html">Alain Delon</a>
</div>
<div class="meta-sub light">
Rôle : Tom Ripley
</div>
</div>
</div>
<div class="card person-card person-card-col">
<figure class="thumbnail">
<a class="xXx thumbnail-container thumbnail-link" href="/personne/

In [104]:
print(type(real))
real_inter = real.find_all('a', class_="meta-title-link")
real_2 = [name_dir.text for name_dir in real_inter]
print(real_2)

<class 'bs4.element.Tag'>
['Alain Delon', 'Marie Laforêt', 'Maurice Ronet', 'Erno Crisa', 'Elvire Popesco', 'Frank Latimore', 'Billy Kearns', 'Romy Schneider']


In [102]:
url = "https://www.allocine.fr/film/fichefilm_gen_cfilm=4916.html"

pattern = r"(?<=gen_cfilm=)\d+(?=\.html)"
match = re.search(pattern, url)

if match:
    film_id = match.group(0)
    url_casting = 'https://www.allocine.fr/film/fichefilm-' + film_id.strip() + '/casting/'
    print(url_casting)
else:
    print("Pas de correspondance trouvée.")

https://www.allocine.fr/film/fichefilm-4916/casting/
